# Tutorial 02: Custom Hamiltonian

Learn how to define custom MFG problems by implementing your own Hamiltonian and problem components.

## Learning Objectives

- Understand the `MFGProblem` abstract class
- Implement custom Hamiltonians with congestion effects
- Define terminal costs and initial densities
- Work with the fundamental MFG equations

## Mathematical Formulation

We'll solve a crowd evacuation problem where agents want to reach a safe zone while avoiding congestion:

**HJB equation** (backward in time):
$$-\frac{\partial u}{\partial t} + H(x, p, m) = 0$$

**Fokker-Planck equation** (forward in time):
$$\frac{\partial m}{\partial t} - \sigma^2 \frac{\partial^2 m}{\partial x^2} + \frac{\partial}{\partial x}\left(m \frac{\partial H}{\partial p}\right) = 0$$

**Hamiltonian** with congestion:
$$H(x, p, m) = \frac{1}{2}|p|^2 + \kappa \cdot m \cdot |p|^2$$

**Time estimate**: 15 minutes

## Step 1: Import Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from mfg_pde import MFGProblem
from mfg_pde.core import MFGComponents
from mfg_pde.core.hamiltonian import HamiltonianBase
from mfg_pde.geometry import TensorProductGrid
from mfg_pde.geometry.boundary import no_flux_bc

## Step 2: Define Custom Hamiltonian Class

We subclass `HamiltonianBase` and implement:
- `__call__()`: The Hamiltonian value H(x, m, p, t)
- `dp()`: Derivative of H with respect to momentum (for optimal control)
- `dm()`: Derivative of H with respect to density (for HJB coupling)

In [ ]:
class CongestionHamiltonian(HamiltonianBase):
    """Hamiltonian with congestion-dependent control cost.

    H(x, m, p, t) = (1/2)|p|^2 + kappa * m * |p|^2

    Standard control cost plus congestion-dependent increase.
    High density m makes control more expensive.
    """

    def __init__(self, congestion_strength: float = 1.0):
        super().__init__()
        self.congestion_strength = congestion_strength

    def __call__(self, x, m, p, t=0.0):
        """H = (1/2)|p|^2 + kappa * m * |p|^2"""
        standard_cost = 0.5 * p**2
        congestion_cost = self.congestion_strength * m * p**2
        return standard_cost + congestion_cost

    def dp(self, x, m, p, t=0.0):
        """dH/dp = p + 2 * kappa * m * p"""
        return p + 2 * self.congestion_strength * m * p

    def dm(self, x, m, p, t=0.0):
        """dH/dm = kappa * |p|^2"""
        return self.congestion_strength * p**2


def create_evacuation_problem(congestion_strength: float = 1.0) -> MFGProblem:
    """Create crowd evacuation MFG problem.

    Args:
        congestion_strength: kappa parameter for congestion coupling

    Returns:
        Configured MFGProblem ready to solve
    """
    # Geometry: 1D domain [0, 1] with 60 points
    geometry = TensorProductGrid(
        bounds=[(0.0, 1.0)],
        Nx_points=[60],
        boundary_conditions=no_flux_bc(dimension=1),
    )

    # Custom Hamiltonian with congestion
    hamiltonian = CongestionHamiltonian(congestion_strength=congestion_strength)

    # Initial density: uniform on [0.1, 0.9]
    def initial_density(x):
        density = np.where((x >= 0.1) & (x <= 0.9), 1.0, 0.0)
        # Normalize (approximate, exact normalization done internally)
        return density / max(np.sum(density) * 0.02, 1e-10)

    # Terminal cost: agents want to reach x = 1 (safe zone)
    def terminal_cost(x):
        return (x - 1.0) ** 2

    components = MFGComponents(
        hamiltonian=hamiltonian,
        m_initial=initial_density,
        u_final=terminal_cost,
    )

    return MFGProblem(
        geometry=geometry,
        T=1.0,
        Nt=60,
        diffusion=0.05,
        components=components,
    )


print("Custom Hamiltonian class and problem factory defined!")

## Step 3: Solve Without Congestion

First, solve with no congestion (κ = 0) to establish baseline behavior.

In [ ]:
problem_no_congestion = create_evacuation_problem(congestion_strength=0.0)
print("Solving without congestion (kappa = 0)...\n")
result_no_congestion = problem_no_congestion.solve(verbose=True)

print(f"\nConverged: {result_no_congestion.converged}")
print(f"Iterations: {result_no_congestion.iterations}")

## Step 4: Solve With Congestion

Now solve with congestion effects (κ = 1.0).

In [ ]:
problem_with_congestion = create_evacuation_problem(congestion_strength=1.0)
print("Solving with congestion (kappa = 1.0)...\n")
result_with_congestion = problem_with_congestion.solve(verbose=True)

print(f"\nConverged: {result_with_congestion.converged}")
print(f"Iterations: {result_with_congestion.iterations}")

## Step 5: Compare Results

Visualize how congestion affects the solution.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Get grid coordinates
x = np.linspace(0, 1, problem_no_congestion.geometry.get_grid_shape()[0])
t = np.linspace(0, problem_no_congestion.T, problem_no_congestion.Nt + 1)

# Density at t=0 (initial)
axes[0, 0].plot(x, result_no_congestion.M[0, :], "b-", label="No congestion", linewidth=2)
axes[0, 0].plot(x, result_with_congestion.M[0, :], "r--", label="With congestion", linewidth=2)
axes[0, 0].set_xlabel("x")
axes[0, 0].set_ylabel("Density m(0, x)")
axes[0, 0].set_title("Initial Density")
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Density at t=T (final)
axes[0, 1].plot(x, result_no_congestion.M[-1, :], "b-", label="No congestion", linewidth=2)
axes[0, 1].plot(x, result_with_congestion.M[-1, :], "r--", label="With congestion", linewidth=2)
axes[0, 1].set_xlabel("x")
axes[0, 1].set_ylabel("Density m(T, x)")
axes[0, 1].set_title("Final Density")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Value function at t=0
axes[1, 0].plot(x, result_no_congestion.U[0, :], "b-", label="No congestion", linewidth=2)
axes[1, 0].plot(x, result_with_congestion.U[0, :], "r--", label="With congestion", linewidth=2)
axes[1, 0].set_xlabel("x")
axes[1, 0].set_ylabel("Value u(0, x)")
axes[1, 0].set_title("Initial Value Function")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Density evolution (with congestion)
X, T_mesh = np.meshgrid(x, t)
contour = axes[1, 1].contourf(X, T_mesh, result_with_congestion.M, levels=20, cmap="viridis")
axes[1, 1].set_xlabel("x")
axes[1, 1].set_ylabel("t")
axes[1, 1].set_title("Density Evolution (With Congestion)")
plt.colorbar(contour, ax=axes[1, 1])

plt.tight_layout()
plt.show()

## Step 6: Analyze Congestion Effects

In [ ]:
print("=" * 70)
print("CONGESTION EFFECTS ANALYSIS")
print("=" * 70)
print()

# Compare final densities at target x = 1.0
m_final_no_cong = result_no_congestion.M[-1, -1]
m_final_with_cong = result_with_congestion.M[-1, -1]

print("Final density at target (x=1.0):")
print(f"  No congestion:   m(T, 1.0) = {m_final_no_cong:.4f}")
print(f"  With congestion: m(T, 1.0) = {m_final_with_cong:.4f}")
print()

# Compare value functions (cost to reach target)
u_initial_no_cong = np.mean(result_no_congestion.U[0, :])
u_initial_with_cong = np.mean(result_with_congestion.U[0, :])

print("Average initial cost:")
print(f"  No congestion:   u(0, x) = {u_initial_no_cong:.4f}")
print(f"  With congestion: u(0, x) = {u_initial_with_cong:.4f}")
print(f"  Increase: {(u_initial_with_cong - u_initial_no_cong) / u_initial_no_cong * 100:.1f}%")
print()

print("Interpretation:")
print("  - Congestion increases optimal cost (higher u)")
print("  - Agents spread out more to avoid crowding")
print("  - Final density at target is reduced")

## Summary

### What You Learned

1. How to subclass `HamiltonianBase` to define custom Hamiltonians
2. How to implement congestion effects via the `dm()` method
3. How to use `MFGComponents` to bundle problem components
4. How congestion affects optimal behavior and equilibrium

### Key Concepts

- **HamiltonianBase**: Base class for custom Hamiltonian implementations
- **Congestion coupling**: Higher density makes control more expensive (dm term)
- **MFGComponents**: Bundles hamiltonian, m_initial, u_final together
- **problem.solve()**: Primary API for solving MFG problems

### Mathematical Insight

The congestion term kappa * m * |p|^2 in the Hamiltonian creates a feedback loop:
- High density m increases control cost
- Agents slow down in crowded regions
- This spreads out the density, reducing congestion

### Next Steps

Proceed to **Tutorial 03: 2D Geometry** to learn about multi-dimensional problems.